In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from bs4 import BeautifulSoup as bs
import lxml
import glob
import re
import json

In [2]:
def clean_section(s):
    has_paragraph = False
    place_title = ""
    clean_lines = []
    lines = s.split("\n")
    for line in lines:
            
        if re.search("s[0-9]+\">", line): # If line does not contain sth like: s1">
            line = re.sub("s[0-9]+\">(.*)", r"\1", line)
            line = line.strip()
            if line:
                clean_lines.append(line)
        if re.search("</section>", line): # If line does not contain sth like: s1">
            line = line.strip()
            line = re.sub("(.*)</section>", r"\1", line)
            if line:
                clean_lines.append(line)
        else:
            clean_lines.append(line)
                
    lines = clean_lines
    clean_lines = []
                
    for line in lines:
        line = line.strip()
        if not re.match("s[0-9]+\">", line) and not line == "</section>": # If line does not have this shape: s1">
            if line.startswith("<para"):
                has_paragraph = True
                clean_lines.append(line)
            else:
                if has_paragraph == False:
                    place_title = line
                else:
                    clean_lines.append(line)
    clean_text = " ".join(clean_lines)
    # clean_text = bs(clean_text).get_text() # Return clean text
    return place_title, clean_text

In [7]:
regex_section = r"s[0-9]+\">.*"

entry_id = 0
bho_dict = dict()
for path in glob.glob("/resources/bho/*/*"):
    # Read the XML file
    with open(path, "r", encoding="ISO-8859-1") as file:
        # Read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        content = "".join(content)
        bs_content = bs(content, "lxml")
        report_title = bs_content.html.body.find("title").getText()
        if report_title not in ["Preface", "Index", "Appendix", "Elizabeth", "Appendix 2"]:
            sectext = str(bs_content.html.body)
            sections = sectext.split("<section id=\"")
            for s in sections:
                s = s.strip()
                if re.match(regex_section, s):
                    title, text = clean_section(s)
                    if text:
                        entry_id += 1
                        bho_dict[entry_id] = {"report_title": report_title,
                                              "place_name": title,
                                              "description": text,
                                              "filepath": path}

In [8]:
with open('bho_parsed.json', 'w') as f:
    json.dump(bho_dict, f)